In [1]:
import pandas as pd
import numpy as np
import mlflow

# experiments
from experiments.noaa.machine_learning import NOAAMLTraining, FeatureExtractionExperiment
from experiments.noaa.deterministic import NOAADeterministicExperiment
from experiments.noaa.kriging_experiment import NOAAKrigingExperiment

In [17]:
mlflow_experiment = mlflow.get_experiment_by_name(NOAADeterministicExperiment.experiment_name)
df = mlflow.search_runs([mlflow_experiment.experiment_id]).sort_values(["start_time"], ascending=False)
metric_cols =  df.columns[df.columns.str.contains("metrics")].tolist()
param_cols = ["rbf","params.epsilon","params.eval_set", "tags.config"]
eps_results_df = (
    df.dropna(how="all",axis=1)
    .loc[(df["tags.config"].str.contains("rbf"))&(df.start_time>"2022-06-19")]
    .assign(rbf=lambda x: x["tags.config"].str.replace("rbf_(.*)_eps_(.*)", r"\1", regex=True))
    .loc[:,param_cols+sorted(metric_cols)]
    .groupby(["params.eval_set","rbf"])
    .apply(lambda x: x.sort_values("metrics.mae",ascending=True))
    .drop(columns=["params.eval_set","rbf"])
    .dropna(subset=["metrics.rmse"])
)
eps_results_df

params.epsilon  \
params.eval_set rbf                                            
"set1"          gaussian                   54            0.5   
                                           63            1.0   
                                           50            0.1   
                                           62            1.5   
                                           61            2.0   
                                           44            5.0   
                                           67           0.05   
                                           55           0.01   
                inverse_multiquadric       46            1.0   
                                           59            0.5   
                                           51            1.5   
                                           45            2.0   
                                           47            0.1   
                                           58            5.0   
                                           56           0.05   
                                           48           0.01   
                multiquadric               60            5.0   
                                           49            2.0   
                                           64            1.5   
                                           53            1.0   
                                           66            0.5   
                                           65            0.1   
                                           52           0.05   
                                           57           0.01   
                rbf_cubic_set1             29            1.0   
                                           36            1.0   
                rbf_thin_plate_spline_set1 25            1.0   
                                           39            1.0   
"set2"          gaussian                   17            0.5   
                                           18            1.0   
                                           4             0.1   
                                           20           0.05   
                inverse_multiquadric       5             0.5   
                                           1             1.0   
                                           7             2.0   
                                           15           0.05   
                multiquadric               19            5.0   
                                           9             0.5   
                                           3             1.0   
                                           13           0.05   
                rbf_cubic_set2             35            1.0   
                                           38            1.0   
                rbf_thin_plate_spline_set2 32            1.0   
                                           37            1.0   
"set3"          gaussian                   16            0.5   
                                           8             1.0   
                                           22            0.1   
                                           12           0.05   
                inverse_multiquadric       2             0.5   
                                           14            1.0   
                                           10            2.0   
                                           6            0.05   
                multiquadric               21            5.0   
                                           23            0.5   
                                           0             1.0   
                                           11           0.05   
                rbf_cubic_set3             26            1.0   
                rbf_thin_plate_spline_set3 31            1.0   

                                                                          tags.config  \
params.eval_set rbf                                                                     
"set1"          gaussian   

In [2]:
available_configs = NOAADeterministicExperiment.config.config.keys()
mlflow_experiment = mlflow.get_experiment_by_name(NOAADeterministicExperiment.experiment_name)
df = mlflow.search_runs([mlflow_experiment.experiment_id]).sort_values(["start_time"], ascending=False)
metric_cols =  df.columns[df.columns.str.contains("metrics")].tolist()
param_cols = ["params.epsilon","params.eval_set", "tags.config"]
without_eps = ["thin_plate_spline", "cubic", "idw","linear"]
results_by_set = (
    df.dropna(how="all",axis=1)
    .loc[(df.start_time>"2022-06-19 18:00:00")&(df["tags.config"].isin(available_configs))]
    # .assign(rbf=lambda x: x["tags.config"].str.replace("rbf_(.*)_eps_(.*)", r"\1", regex=True))
    .loc[:,param_cols+sorted(metric_cols)]
    .rename(columns={col: col.replace("metrics.","").replace("params.","") for col in df.columns})
    .rename(columns={"eval_set":"eval_area", "tags.config":"algorithm"})
    .assign(eval_area = lambda df: df.eval_area.str.replace("set","Area "))
    .replace(
        {"_set.": "", "linear":"linear_barycentric", '"':'', "eps_.*":"","_": " "},
        value=None,
        regex=True
    )
    .assign(epsilon = lambda df: df.apply(lambda row: row.epsilon if not any(a in row.algorithm for a in without_eps) else "-", axis=1))
    .drop_duplicates(subset=["eval_area","algorithm"], keep="first")
    .groupby(["eval_area"])
    .apply(lambda x: x.sort_values("rmse",ascending=True))
    .reset_index(level=-1,drop=True)
    .set_index("algorithm", append=True)
    .drop(columns=["eval_area","time_to_evaluate"])#, "params.epsilon"])
    .dropna(subset=["rmse"])
    .sort_index(axis=1)    
)
# results_by_set = results_by_set[~results_by_set.index.get_level_values("algorithm").str.contains("time")]
results_by_set

epsilon       mae        r2      rmse  \
eval_area algorithm                                                         
Area 1    linear barycentric time         -  0.204399  0.772895  0.312554   
          rbf cubic                       -  0.207246  0.772344  0.314870   
          linear barycentric              -  0.203498  0.770427  0.315330   
          rbf thin plate spline         1.0  0.207805  0.768965  0.317198   
          rbf multiquadric              1.0  0.209071  0.767002  0.318542   
          rbf inverse multiquadric      0.5  0.209560  0.765577  0.319515   
          idw                             -  0.223756  0.757033  0.325286   
          rbf gaussian                  0.5  0.216872  0.750819  0.329419   
Area 2    linear barycentric              -  0.200597  0.769605  0.306951   
          rbf cubic                       -  0.212199  0.773006  0.315696   
          rbf thin plate spline         1.0  0.214775  0.771197  0.316951   
          rbf multiquadric              1.0  0.217212  0.758810  0.321622   
          rbf inverse multiquadric      0.5  0.218545  0.758040  0.322136   
          rbf gaussian                  0.5  0.229185  0.731678  0.339231   
          idw                             -  0.272814  0.670479  0.380367   
Area 3    linear barycentric              -  0.204607  0.776401  0.317298   
          rbf thin plate spline         1.0  0.225516  0.763210  0.342343   
          rbf multiquadric              1.0  0.228172  0.759951  0.344691   
          rbf cubic                       -  0.226131  0.755461  0.347900   
          rbf inverse multiquadric      0.5  0.232850  0.748634  0.352723   
          rbf gaussian                  0.5  0.269597  0.660226  0.410086   
          idw                             -  0.325760  0.586215  0.452551   

                                     time_to_eval  
eval_area algorithm                                
Area 1    linear barycentric time      122.803654  
          rbf cubic                    574.081649  
          linear barycentric           571.922491  
          rbf thin plate spline        564.232491  
          rbf multiquadric             222.836038  
          rbf inverse multiquadric     227.523670  
          idw                          572.280333  
          rbf gaussian                 227.593938  
Area 2    linear barycentric           411.565576  
          rbf cubic                    421.514870  
          rbf thin plate spline        403.408097  
          rbf multiquadric            3248.535419  
          rbf inverse multiquadric    3259.892989  
          rbf gaussian                3239.657012  
          idw                          396.922902  
Area 3    linear barycentric           456.018855  
          rbf thin plate spline        487.492096  
          rbf multiquadric            2180.374151  
          rbf cubic                    477.160671  
          rbf inverse multiquadric    2222.399592  
          rbf gaussian                2219.686955  
          idw                          440.508804

In [ ]:
print(results_by_set.round(2).to_latex(index=True))

\begin{tabular}{lllrrrr}
\toprule
       &     & epsilon &   mae &    r2 &  rmse &  time\_to\_eval \\
eval\_area & algorithm &         &       &       &       &               \\
\midrule
Area 1 & rbf cubic &       - &  0.21 &  0.77 &  0.31 &        574.08 \\
       & linear barycentric &       - &  0.20 &  0.77 &  0.32 &        571.92 \\
       & rbf thin plate spline &     1.0 &  0.21 &  0.77 &  0.32 &        564.23 \\
       & idw &       - &  0.22 &  0.76 &  0.33 &        572.28 \\
Area 2 & linear barycentric &       - &  0.20 &  0.77 &  0.31 &        411.57 \\
       & rbf cubic &       - &  0.21 &  0.77 &  0.32 &        421.51 \\
       & rbf thin plate spline &     1.0 &  0.21 &  0.77 &  0.32 &        403.41 \\
       & rbf multiquadric  &     1.0 &  0.22 &  0.76 &  0.32 &       3248.54 \\
       & rbf inverse multiquadric  &     0.5 &  0.22 &  0.76 &  0.32 &       3259.89 \\
       & rbf gaussian  &     0.5 &  0.23 &  0.73 &  0.34 &       3239.66 \\
       & idw &       - &  0.2

In [4]:
import subprocess
available_configs = NOAADeterministicExperiment.config.config.keys()
c = 0
for config_name in available_configs: 
    # if "set3" in config_name and "eps" in config_name:
    if config_name in df.loc[(df.start_time>"2022-06-19 18:00:00")&(df["tags.config"].isin(available_configs)), "tags.config"].values:
        # print(config_name)
        continue
    c += 1
    print(config_name)
    experiment_name = NOAADeterministicExperiment.__name__
    eval_frac = "0.2" if "set1" in config_name else "0.1"
    p = subprocess.Popen(["python", "-m", "experiments", experiment_name , config_name, f"--eval_frac={eval_frac}"])
    print(f"Started experiment {config_name} on process {p.pid}")
    if c%5==0:
        p.communicate()

linear_time_set2
Started experiment linear_time_set2 on process 14472
linear_time_set3
Started experiment linear_time_set3 on process 14473
rbf_time_cubic_set1
Started experiment rbf_time_cubic_set1 on process 14474
rbf_time_cubic_set2
Started experiment rbf_time_cubic_set2 on process 14475
rbf_time_cubic_set3
Started experiment rbf_time_cubic_set3 on process 14476
Attempting to run experiment NOAADeterministicExperiment with config rbf_time_cubic_set3...
Running experiment NOAADeterministicExperiment with experiment_config='rbf_time_cubic_set3' and params: {'eval_frac': 0.1}
Attempting to run experiment NOAADeterministicExperiment with config linear_time_set3...
Running experiment NOAADeterministicExperiment with experiment_config='linear_time_set3' and params: {'eval_frac': 0.1}
Attempting to run experiment NOAADeterministicExperiment with config rbf_time_cubic_set2...
Running experiment NOAADeterministicExperiment with experiment_config='rbf_time_cubic_set2' and params: {'eval_frac'

INFO:NOAADeterministicExperiment:Filtering data on area POLYGON ((-45 12.282308, -45 35.55, -98.251934 35.55, -98.251934 12.282308, -45 12.282308))
INFO:NOAADeterministicExperiment:Filtering data on area POLYGON ((-45 12.282308, -45 35.55, -98.251934 35.55, -98.251934 12.282308, -45 12.282308))
INFO:NOAADeterministicExperiment:Filtered data has shapes (2543380, 14) and (333, 4)
INFO:NOAADeterministicExperiment:Filtered data has shapes (2543380, 14) and (333, 4)
INFO:NOAADeterministicExperiment:Filtering data on area POLYGON ((-80 18.930645, -97.806644 18.930645, -97.806644 30.366655, -82.836914 30.366655, -82.836914 35.55, -68.115234 35.55, -68.115234 25.799891, -80 25.799891, -80 18.930645))
INFO:NOAADeterministicExperiment:Filtered data has shapes (1760006, 14) and (232, 4)
INFO:NOAADeterministicExperiment:Filtering data on area POLYGON ((-80 18.930645, -97.806644 18.930645, -97.806644 30.366655, -82.836914 30.366655, -82.836914 35.55, -68.115234 35.55, -68.115234 25.799891, -80 25.7

Split test obtained with dataset(s)=['buoys_data'] and shape(s)=[(80437, 14)]
Split test obtained with dataset(s)=['buoys_data'] and shape(s)=[(116336, 14)]
Split test obtained with dataset(s)=['buoys_data'] and shape(s)=[(116336, 14)]
Split test obtained with dataset(s)=['buoys_data'] and shape(s)=[(122017, 14)]


INFO:NOAADeterministicExperiment:Interpolator is temporal?: True
INFO:NOAADeterministicExperiment:A random subset of 20.0% of the test data will be used for evaluation
fitting interpolator at eval times...:   0%|          | 6/13179 [00:04<2:57:35,  1.24it/s]

Split test obtained with dataset(s)=['buoys_data'] and shape(s)=[(122017, 14)]


fitting interpolator at eval times...:   0%|          | 10/13179 [00:06<2:31:41,  1.45it/s]INFO:NOAADeterministicExperiment:Interpolator is temporal?: True
INFO:NOAADeterministicExperiment:A random subset of 10.0% of the test data will be used for evaluation
fitting interpolator at eval times...:   2%|▏         | 163/7967 [00:29<28:01,  4.64it/s]INFO:NOAADeterministicExperiment:Interpolator is temporal?: True
INFO:NOAADeterministicExperiment:A random subset of 10.0% of the test data will be used for evaluation
fitting interpolator at eval times...:  83%|████████▎ | 10895/13179 [2:02:47<16:34,  2.30it/s]

------

In [15]:
config = FeatureExtractionExperiment("set3").get_config()
train_df = pd.concat(
    [pd.read_parquet(f"{config.output.train_dir}/{year}.parquet") for year in range(2011,2022)],
    axis=0).sort_index()
test_df = pd.concat(
    [pd.read_parquet(f"{config.output.eval_dir}/{year}.parquet") for year in range(2011,2022)],
    axis=0).sort_index()

In [16]:
train_df.distance_to_shore.isnull().sum()

AttributeError: 'DataFrame' object has no attribute 'distance_to_shore'